# _**Lorenz Attractor**_

#### The Lorenz attractor is a set of chaotic solutions to the Lorenz system, a system of three nonlinear differential equations initially developed to model atmospheric convection. The system exhibits sensitive dependence on initial conditions, which is a hallmark of chaotic behavior. 
The equations governing the Lorenz system are:
$$
\begin{cases}
\dot{x}​=σ(y−x), \\
\dot{y}=x(ρ−z)−y, \\
\dot{z}=xy−βz; \\
\end{cases}
$$
where $x$, $y$, and $z$ represent the system's state variables, and $\sigma$, $\rho$, and $\beta$ are parameters. Typically, $\sigma$ represents the Prandtl number, $\rho$ is the Rayleigh number, and $\beta$ is a geometric factor. The Lorenz attractor emerges when these parameters are set to specific values, such as $\sigma$=10, $\rho$=28, and $\beta$=$\frac83$​, revealing the system's chaotic nature and its characteristic butterfly-shaped trajectory in phase space.

### Equilibrium Points
$$
\begin{cases}
\dot{x}​=σ(y−x)=0, \\
\dot{y}=x(ρ−z)−y=0, \\
\dot{z}=xy−βz=0; \\
\end{cases}
\implies
\begin{cases}
x=y, \\
x = \pm \sqrt{βz},\\
x(ρ−z-1)=0; \\
\end{cases}
\implies
\begin{cases}
x^*_1 = (0,0,0), \\
x^*_2= (\sqrt{β(ρ-1)}, \; \sqrt{β(ρ-1)},\; ρ-1),  \quad ρ\geq 1;\\
x^*_3= (-\sqrt{β(ρ-1)}, \; -\sqrt{β(ρ-1)},\; ρ-1),  \quad ρ\geq 1.\\
\end{cases}
$$

In [5]:
import nbformat
import numpy as np
import plotly.graph_objects as go
import plotly.express as px


def lorenz(t, state, sigma, rho, beta):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return np.array([dxdt, dydt, dzdt])

def RK4(f, initial_state, t0, tf, dt, args):
    def f_(t,state, args=args):
        return f(t, state, *args)
    t = np.arange(t0, tf, dt)
    n = len(t)
    states = np.zeros((n, len(initial_state)))
    states[0] = initial_state
    for i in range(n - 1):
        state= states[i]
        k1 = f_(t[i], state)
        k2 = f_(t[i] + dt / 2, state + dt / 2 * k1)
        k3 = f_(t[i] + dt / 2, state + dt / 2 * k2)
        k4 = f_(t[i] + dt, state + dt * k3)
        states[i + 1] = states[i] + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return states

def solve_ivp(f, t_span, y0, args, t_eval):
    t0, tf = t_span 
    dt = t_eval[1] - t_eval[0]
    return RK4(f, y0, t0, tf, dt, args)

def plot_lorenz_attractor(
    sigma: float = 10.0,
    rho: float = 28.0,
    beta: float = 8.0 / 3.0,
    initial_state: np.ndarray = np.array([1.0, 0.0, 0.0]),
    t_start: float = 0.0,
    t_end: float =400.0,
    plots: int = 1,
    ERROR: float = 0.001,
    cmap: str = "GnBu",
    plot_thickness: int = 2,
    color_offset: float = 0.1,
    fig_size: tuple = (1350, 900),
):
    plots = int(plots)
    if plots < 1:
        raise ValueError("Number of plots must be at least 1")

    t_eval = np.linspace(t_start, t_end, 1000 * int(t_end - t_start))

    fig = go.Figure()
    color_values = np.linspace(color_offset, 1- color_offset, plots)

    for plot_num in range(plots):
        perturbed_state = initial_state * (1 + (ERROR * plot_num))  # Small perturbation

        solution = solve_ivp(
            lorenz,
            [t_start, t_end],
            perturbed_state,
            args=(sigma, rho, beta),
            t_eval=t_eval,
        )
        color_value = 0.5 if plots==1 else color_values[plot_num]  # Map plot_num to a value between 0 and 1

        # Get the corresponding color from the colorscale
        color = px.colors.sample_colorscale(cmap, [color_value])[0]

        x, y, z = solution.T

        fig.add_trace(
            go.Scatter3d(
                x=x,
                y=y,
                z=z,
                mode="lines",
                line=dict(
                    color=color,
                    width=plot_thickness,
                ),
                name=f"Initial Condition {plot_num + 1}",
            )
        )

    fig.update_traces(opacity=0.4)

    fig.update_layout(
        title=(
            f"Lorenz Attractor - Two Initial Conditions (ERROR:{ERROR*100}%)"
            if plots > 1
            else "Lorenz Attractor - One Initial Condition"
        ),
        scene=dict(
            xaxis=dict(
                backgroundcolor="black",
                gridcolor="white",
                showbackground=True,
                zerolinecolor="white",
            ),
            yaxis=dict(
                backgroundcolor="black",
                gridcolor="white",
                showbackground=True,
                zerolinecolor="white",
            ),
            zaxis=dict(
                backgroundcolor="black",
                gridcolor="white",
                showbackground=True,
                zerolinecolor="white",
            ),
            camera=dict(
                up=dict(x=0, y=0, z=1),
                center=dict(x=0, y=0, z=0),
                eye=dict(x=2.5, y=0.1, z=0.1),
            ),
        ),
        margin=dict(r=10, l=10, b=10, t=10),
        paper_bgcolor="black",
        plot_bgcolor="black",
        width=fig_size[0],
        height=fig_size[1],
        font=dict(color="white"),
        showlegend=True,
    )

    fig.show()


 ### To run the plots change the veriable run to 1 or True.

In [1]:
run = 0 # Change this to 1 to run the code

In [7]:
if run:
    plot_lorenz_attractor(plots=1, plot_thickness=3, cmap='Tealgrn', fig_size=(900, 900))

In [8]:
if run:
    plot_lorenz_attractor(plots=2, plot_thickness=1.7, cmap='Plasma', color_offset=0.2, fig_size=(900, 900))

## **Observation**
#### We can quite clearly see that although the error was not significant at all (a mere 0.1%), it nonetheless had a substantial effect on the behavior and evolution of the system. This is a key feature of deterministic chaotic systems: the smallest changes in initial conditions lead to dramatically different outcomes. The 'deterministic' part means that although the system seems to exhibit random and aperiodic behavior, the path that the system will take, given the same initial conditions, will always stay the same.

The animation below demonstrates how little changes in initial conditions can lead to vastly different trajectories. The 3 trajectories start off very close to each other, but quickly diverge and end up in completely different regions of phase space. This is a hallmark of chaotic systems and is known as the butterfly effect. The initial conditions are as follows: 
- **Red** represents the trajectory with initial conditions $$x_0 = \hat{i};$$
-  **Orange** represents the trajectory with initial conditions $$x_1 = \hat{i} + 0.01\hat{j};$$
-   **Blue** represents the trajectory with initial conditions $$x_2 = \hat{i} + 0.01\hat{k}.$$ 


<video width="1000" height="600" controls>
  <source src="LorenzAttractor.mp4" type="video/mp4">
</video>

In [ ]:
# Code that genrated the animation

from manim import *
import numpy as np

def lorenz(t, state, sigma, rho, beta):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return np.array([dxdt, dydt, dzdt])

def RK4(f, initial_state, t0, tf, dt, args):
    def f_(t,state, args=args):
        return f(t, state, *args)
    t = np.arange(t0, tf, dt)
    n = len(t)
    states = np.zeros((n, len(initial_state)))
    states[0] = initial_state
    for i in range(n - 1):
        state= states[i]
        k1 = f_(t[i], state)
        k2 = f_(t[i] + dt / 2, state + dt / 2 * k1)
        k3 = f_(t[i] + dt / 2, state + dt / 2 * k2)
        k4 = f_(t[i] + dt, state + dt * k3)
        states[i + 1] = states[i] + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return states

def solve_ivp(f, t_span, y0, args, t_eval):
    t0, tf = t_span 
    dt = t_eval[1] - t_eval[0]
    return RK4(f, y0, t0, tf, dt, args)

class LorenzAttractor(ThreeDScene):
    def construct(self):
        # Lorenz system parameters
        sigma = 10
        rho = 28
        beta = 8 / 3
        initial_states = np.array([[1, 0, 0],
                                   [1, 0.01, 0],
                                   [1, 0, 0.01]])

        # Create the 3D axes with wider ranges
        axes = ThreeDAxes(
            x_range=[-1, 1, 0.5],
            y_range=[-1, 1, 0.5],
            z_range=[-1, 1, 0.5]
            )
        
        t0, tf = 0, 40
        t_eval = np.linspace(t0, tf, (tf - t0) * 100)
        t_span = [t0, tf]
        curves = VGroup()
        # Iterate over initial states to create 3 curves
        for initial_state, color in zip(initial_states, [RED, ORANGE, BLUE]):
            # Solve the system using RK4
            states = solve_ivp(lorenz, t_span, initial_state, args=(sigma, rho, beta), t_eval=t_eval)
            states = states / 50
            
            # Convert the states to 3D points
            curve = [axes.c2p(x, y, z) for x, y, z in states]
            curve_obj = VMobject(stroke_width=2, stroke_opacity=0.4, color=color).set_points_smoothly(curve).set_color(color)
            curves.add(curve_obj)
            
        dots = VGroup(
            *(Dot3D(color=color) for color in [RED, ORANGE, BLUE])
        )
        
        def update_dots(dots):
            for dot, curve_obj in zip(dots, curves):
                dot.move_to(curve_obj.get_end())
           
        dots.add_updater(update_dots)
        
        # Set up camera angle

        self.wait(1)
        self.set_camera_orientation(phi=75 * DEGREES, theta=-30 * DEGREES)

        # Add the axes and curves to the scene
        self.add(axes, dots)
        self.play(
            
            *[Create(curve_obj, rate_func=linear) for curve_obj in curves],
            run_time=tf - t0
        )
        # Begin ambient camera rotation to visualize the attractor from different angles
        self.begin_ambient_camera_rotation(rate=0.2)

        # # Wait for a few seconds at the end to view the final attractor
        self.wait(15)

With this code in a file script.py we can run the following command to generate the video LorenzAttractor.mp4
```bash
manim -pqh script.py LorenzAttractor
```
The `-pqh` flag is used to render the video in a higher quality. For lower quality and faster rendering, use the `-pql` flag.